# 1. Multiple categories

In the last mission, we calculated a chi-squared value indicating how the observed frequencies in a single categorical column, such as sex, varied from the US population as a whole.

In this mission, we'll look how to make this same technique applicable to cross tables, that show how two categorical columns interact. For instance, here's a table showing the relationship between sex and high_income:

|||Male|Female|Totals|
|--|----|----|-----|----|
||>50K|6662|1179|7841|
|Income|<=50K|15128|9592|24720|
||Totals|21790|10771|32561|

On looking at this diagram, you might see a pattern between sex and high_income. But it's hard to immediately quantify that pattern, and tell if it's significant. We can apply the chi-squared test (also known as the chi-squared test of association) to figure out if there's a statistically significant correlation between two categorical columns.

# 2. Calculating expected values

In the single category chi-squared test, we find expected values from other data sets, and then compare with our own observed values. In a multiple category chi-squared test, we calculate expected values across our whole dataset. We'll illustrate this by converting our chart from last screen into proportions: 

|||Male|Female|Totals|
|--|----|----|-----|----|
||>50K|.205|.036|.241|
|Income|<=50K|.465|.294|.759|
||Totals|.67|.33|1|


Each cell represents the proportion of people in the data set that fall into the specified categories.

* 20.5% of people in the whole data set are males earning >50k in income.
* 33% of the whole dataset is Female
* 75.9% of the whole dataset earns <=50k.

In [1]:
males_over50k = .67 * .241 * 32561
males_under50k = .67 * .759 * 32561
females_over50k = .33 * .241 * 32561
females_under50k = .33 * .759 * 32561

# 3. Calculating chi-squared

Now that we have our expected values, we can calculate the chi-squared value by using the same principle from the previous mission. Here are the steps:

* Subtract the expected value from the observed value.
* Square the difference.
* Divide the squared difference by the expected value.
* Repeat for all the observed and expected values and add up the values.

In [2]:
observed = [6662, 1179, 15128, 9592]
expected = [5257.6, 2589.6, 16558.2, 8155.6]
values = []

for i, obs in enumerate(observed):
    exp = expected[i]
    value = (obs - exp) ** 2 / exp
    values.append(value)

chisq_gender_income = sum(values)

# 4. Finding statistical significance

Now that we've found our chi-squared value, 1520.0, we can use the same technique with the chi-squared sampling distribution from the last mission to find a p-value associated with the chi-squared value. The p-value will tell us whether the difference between the observed and expected values is statistically significant or not.

Rather than construct a sampling distribution again manually, we'll use the scipy.stats.chisquare function that we covered in the last mission.

In [3]:
import numpy as np
from scipy.stats import chisquare

observed = np.array([6662, 1179, 15128, 9592])
expected = np.array([5257.6, 2589.6, 16558.2, 8155.6])

chisq_value, pvalue_gender_income = chisquare(observed, expected)

# 5. Cross tables

We can also scale up the chi-squared test to cases where each category contains more than two possibilities. We'll illustrate this with an example where we look at sex vs race.

Before we can find the chi-squared value, we need to find the observed frequency counts. We can do this using the `pandas.crosstab` function. The crosstab function will print a table that shows frequency counts for two or more columns. 

`import pandas

table = pandas.crosstab(income["sex"], [income["high_income"]])
print(table)`

The above code will print a table showing how many people from income fall into each category of sex and high_income.

The second parameter passed into pandas.crosstab is actually a list -- this parameter can contain more than one item.

In [4]:
import pandas as pd
income=pd.read_csv('income.csv')

In [5]:
import pandas
table = pandas.crosstab(income["sex"], [income["race"]])
print(table)

race      Amer-Indian-Eskimo   Asian-Pac-Islander   Black   Other   White
sex                                                                      
 Female                  119                  346    1555     109    8642
 Male                    192                  693    1569     162   19174


# 6. Finding expected values

Now that we have the observed frequency counts, we can generate the expected values. We can use the [scipy.stats.chi2_contingency](http://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.chi2_contingency.html) function to do this. The function takes in a cross table of observed counts, and returns the chi-squared value, the p-value, the degrees of freedom, and the expected frequencies. 

In [6]:
import pandas
from scipy.stats import chi2_contingency

table = pandas.crosstab(income["sex"], [income["race"]])
chisq_value, pvalue_gender_race, df, expected = chi2_contingency(table)

# 7. Caveats

Now that we've learned the chi-squared test, you should be able to figure out if the association between two columns of categorical data is statistically significant or not. There are a few caveats to using the chi-squared test that are important to cover, though:

* Finding that a result isn't significant doesn't mean that no association between the columns exists. For instance, if we found that the chi-squared test between the sex and race columns returned a p-value of .1, it wouldn't mean that there is no relationship between sex and race. It just means that there isn't a statistically significant relationship.
* Finding a statistically significant result doesn't imply anything about what the correlation is. For instance, finding that a chi-squared test between sex and race results in a p-value of .01 doesn't mean that the dataset contains too many Females who are White (or too few). A statistically significant finding means that some evidence of a relationship between the variables exists, but needs to be investigated further.
* Chi-squared tests can only be applied in the case where each possibility within a category is independent. For instance, the Census counts individuals as either Male or Female, not both.
* Chi-squared tests are more valid when the numbers in each cell of the cross table are larger. So if each number is 100, great -- if each number is 1, you may need to gather more data.

In this mission, we covered chi-squared tests for multiple categories, and learned how to quickly perform chi-squared tests. We learned when to apply and when not to apply chi-squared tests. Chi-squared tests can be a powerful tool to discover correlations and figure out when anomalies in your data should be investigated further.